# Eager and Lazy APIs

In [1]:
import polars as pl

In [ ]:
%%time
trips = pl.read_parquet("data/taxi/yellow_tripdata_*.parquet")
sum_per_vendor=trips.group_by("VendorID").sum()

income_per_distance_per_vendor = sum_per_vendor.select(
    "VendorID",
    income_per_distance = pl.col("total_amount")/pl.col("trip_distance"),
    )

top_three = income_per_distance_per_vendor.sort(
    by = "income_per_distance", descending=True
).head(3)

: 

haha looks like it was too much data and it killed the kernal in the dev container :D

In [2]:
%%time
trips = pl.scan_parquet("data/taxi/yellow_tripdata_*.parquet")
sum_per_vendor=trips.group_by("VendorID").sum()

income_per_distance_per_vendor = sum_per_vendor.select(
    "VendorID",
    income_per_distance = pl.col("total_amount")/pl.col("trip_distance"),
    )

top_three = income_per_distance_per_vendor.sort(
    by = "income_per_distance", descending=True
).head(3)

top_three.collect()

CPU times: user 1.39 s, sys: 587 ms, total: 1.98 s
Wall time: 1.24 s


VendorID,income_per_distance
i64,f64
1,6.434789
6,5.296493
5,4.731557


very cool the cell above can actually run in the dinky little container, not true for the eager api.
there are 12 parquet files, each one ~ around 30-50mb. so ltos of data. lets actually see how many rows that is...

In [ ]:
schema = trips.collect_schema()
schema

Schema([('VendorID', Int64),
        ('tpep_pickup_datetime', Datetime(time_unit='ns', time_zone=None)),
        ('tpep_dropoff_datetime', Datetime(time_unit='ns', time_zone=None)),
        ('passenger_count', Float64),
        ('trip_distance', Float64),
        ('RatecodeID', Float64),
        ('store_and_fwd_flag', String),
        ('PULocationID', Int64),
        ('DOLocationID', Int64),
        ('payment_type', Int64),
        ('fare_amount', Float64),
        ('extra', Float64),
        ('mta_tax', Float64),
        ('tip_amount', Float64),
        ('tolls_amount', Float64),
        ('improvement_surcharge', Float64),
        ('total_amount', Float64),
        ('congestion_surcharge', Float64),
        ('airport_fee', Float64)])

In [17]:

len(schema)

19

In [18]:

trips.select(pl.len()).collect().item()

39656098

goodness thats a pretty big data set -  39million rows x 19 columns